In [1]:
%load_ext autoreload
# %reload_ext autoreload
%autoreload 2


In [2]:
from methylVA.mnist.dataset import get_methyl_data_loaders

data_id = 0.1
batch_size = 128

train_data_path = f"../data/dimension_reduction/highly_variable_features/train_data_filtered_{data_id}.pkl"
train_metadata_path = f"../data/dimension_reduction/highly_variable_features/train_metadata_with_labels.pkl"
test_data_path = f"../data/dimension_reduction/highly_variable_features/test_data_filtered_{data_id}.pkl"
test_metadata_path = f"../data/dimension_reduction/highly_variable_features/test_metadata_with_labels.pkl"


train_loader, test_loader = get_methyl_data_loaders(
    train_data_path,
    train_metadata_path,
    test_data_path,
    test_metadata_path,
    batch_size=batch_size
)


Found NaN values in the data after conversion.
Found NaN values in the data after conversion.


In [3]:
data_batch, _ = next(iter(train_loader))


num_train_rows = len(train_loader.dataset)
num_test_rows = len(test_loader.dataset)

print("Number of features in each dataset:", data_batch.shape[1])
print("Number of rows in the training dataset:", num_train_rows)
print("Number of rows in the test dataset:", num_test_rows)

Number of features in each dataset: 2605
Number of rows in the training dataset: 33360
Number of rows in the test dataset: 3707


In [4]:
from datetime import datetime

import torch
from torch.utils.tensorboard import SummaryWriter

from methylVA.mnist.model import VAE
from methylVA.mnist.training import train, test

input_dim = data_batch.shape[1]
learning_rate = 1e-3
weight_decay = 1e-2
num_epochs = 100
latent_dim = 32
hidden_dim = 2048
kl_weight = 1.0
name = f'VAE_methyl_data_{data_id}_latent_{latent_dim}_kl_{kl_weight}'



In [5]:

writer_train = SummaryWriter(f'../experiments/{name}/train/{datetime.now().strftime("%Y%m%d-%H%M%S")}')
writer_test = SummaryWriter(f'../experiments/{name}/test/{datetime.now().strftime("%Y%m%d-%H%M%S")}')

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = VAE(input_dim=input_dim, latent_dim=latent_dim, hidden_dim=hidden_dim).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

In [7]:
model

VAE(
  (encoder): Sequential(
    (0): Linear(in_features=2605, out_features=2048, bias=True)
    (1): SiLU()
    (2): Linear(in_features=2048, out_features=1024, bias=True)
    (3): SiLU()
    (4): Linear(in_features=1024, out_features=512, bias=True)
    (5): SiLU()
    (6): Linear(in_features=512, out_features=256, bias=True)
    (7): SiLU()
    (8): Linear(in_features=256, out_features=64, bias=True)
  )
  (softplus): Softplus(beta=1.0, threshold=20.0)
  (decoder): Sequential(
    (0): Linear(in_features=32, out_features=256, bias=True)
    (1): SiLU()
    (2): Linear(in_features=256, out_features=512, bias=True)
    (3): SiLU()
    (4): Linear(in_features=512, out_features=1024, bias=True)
    (5): SiLU()
    (6): Linear(in_features=1024, out_features=2048, bias=True)
    (7): SiLU()
    (8): Linear(in_features=2048, out_features=2605, bias=True)
    (9): Sigmoid()
  )
)

In [8]:
from methylVA.mnist.training import train, test


prev_updates = 0
for epoch in range(num_epochs):
    print(f'Epoch {epoch + 1}/{num_epochs}')
    prev_updates = train(model, train_loader, optimizer, prev_updates, writer=writer_train)
    test(model, test_loader, prev_updates, writer=writer_test)

Epoch 1/100


  2%|▏         | 4/261 [00:00<00:25, 10.01it/s]

Step 0, (N samples: 0), Loss: 1808.9576, (Recon: 1805.5591, KLD: 3.3986), Gradient norm: 8.7981


 40%|███▉      | 104/261 [00:03<00:04, 31.89it/s]

Step 100, (N samples: 12,800), Loss: 1176.6036, (Recon: 1167.6359, KLD: 8.9678), Gradient norm: 164.2641


...

Step 25,900, (N samples: 3,315,200), Loss: 1053.0640, (Recon: 1043.8555, KLD: 9.2085), Gradient norm: 153.3590


 64%|██████▎   | 166/261 [00:05<00:02, 31.89it/s]

Step 26,000, (N samples: 3,328,000), Loss: 1054.3584, (Recon: 1045.2986, KLD: 9.0599), Gradient norm: 275.4611


Testing: 100%|██████████| 29/29 [00:00<00:00, 36.45it/s]

====> Test set loss: 1066.5811, (BCE: 1057.3722, KLD: 9.2089)


In [10]:
def pearson_correlation(original_x, x_hat, mask=None):
    
    # Calculate mean and standard deviation
    mean_x = torch.mean(original_x)
    mean_x_hat = torch.mean(x_hat)
    
    std_x = torch.std(original_x)
    std_x_hat = torch.std(x_hat)
    
    # Calculate covariance
    covariance = torch.mean((original_x - mean_x) * (x_hat - mean_x_hat))
    
    # Calculate Pearson correlation
    correlation = covariance / (std_x * std_x_hat)
    
    return correlation.item()


correlations = []

# Disable gradient calculation for efficiency
with torch.no_grad():
    for batch in train_loader:
        # Unpack the batch to get the input data; adjust if there are labels
        original_x, _ = batch  # Adjust this if your batch contains more items
        original_x = original_x.to(device)  # Move input to device if using GPU

        # Get the model output for the entire batch
        out = model(original_x)
        x_hat_batch = out.x_recon

        # Compute Pearson correlation for each element in the batch
        for i in range(original_x.size(0)):  # Loop over each sample in the batch
            original_sample = original_x[i]  # Select the i-th sample
            x_hat_sample = x_hat_batch[i]    # Select the corresponding output sample

            # Compute Pearson correlation for the sample
            correlation = pearson_correlation(original_sample, x_hat_sample)
            correlations.append(correlation)
            # print(f"Sample {i} Pearson Correlation:", correlation)

# Optionally, compute average correlation across all elements
average_correlation = sum(correlations) / len(correlations)
print("Average Pearson Correlation across all samples:", average_correlation)


Average Pearson Correlation across all samples: 0.9239656174638335
